# Run parcel tracking with tracer

In [2]:
import numpy as np
from datetime import timedelta as delta
from datetime import datetime
import xarray as xr
from parcels import FieldSet, Variable, ParticleSet, JITParticle, AdvectionRK4_3D, ErrorCode
import glob

%matplotlib inline

ModuleNotFoundError: No module named 'parcels'

#### Parameters

In [1]:
year = 2012

land_color = '#a9a7a2'

#### Load files

In [ ]:
# Open particle initialization location file:
ini_loc = xr.open_dataset('/ocean/brogalla/GEOTRACES/parcels/Pb-chapter/western-PC-transect-parcels.nc')
parcel_lons = ini_loc['longitude'].values
parcel_lats = ini_loc['latitude'].values
parcel_deps = ini_loc['depth'].values

#### General setup

In [3]:
# Mesh mask
mesh_mask = '/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc'

# ANHA12 NEMO grid U,V,W files
folder_ANHA12      = '/data/brogalla/ANHA12/'
ANHA12_gridU_files = sorted(glob.glob(f'{folder_ANHA12}ANHA12-EXH006_y{year}m??d??_gridU.nc'))[0:20]
ANHA12_gridV_files = sorted(glob.glob(f'{folder_ANHA12}ANHA12-EXH006_y{year}m??d??_gridV.nc'))[0:20]
ANHA12_gridW_files = sorted(glob.glob(f'{folder_ANHA12}ANHA12-EXH006_5d_gridW_y{year}m??d??.nc'))[0:20]

# ANHA12 NEMO Pb model files
folder_Pb = f'/data/brogalla/run_storage/Pb-new-ini2-20220922/'
files_Pb  = sorted(glob.glob(f'{folder_Pb}ANHA12_EXH006_5d_{year}0101_{year}1231_ptrc_T_{year}*'))[0:20]

# Create timestamps from file dates:
time_stamps = []
for n, file in enumerate(ANHA12_gridU_files):
    dateU  = datetime.strptime(file[36:47],'y%Ym%md%d')
    dateV  = datetime.strptime(ANHA12_gridV_files[n][36:47],'y%Ym%md%d')
    dateW  = datetime.strptime(ANHA12_gridW_files[n][45:56],'y%Ym%md%d')
    datePb = datetime.strptime(files_Pb[n][99:107],'%Y%m%d')
    
    if (dateU != dateV) | (dateU != dateW) | (dateU != datePb):
        print('Dates not equal!')
    
    # double-check that the date is the same for U, V, and W files
    time_stamps.append(np.datetime64(f'{dateU.year:02}-{dateU.month:02}-{dateU.day:02}'))

timestamps = np.expand_dims(np.array(time_stamps), axis=1)

In [4]:
filenames = {'U': {'lon': mesh_mask   , 'lat': mesh_mask  , 'depth': ANHA12_gridW_files[0], 'data': ANHA12_gridU_files},
             'V': {'lon': mesh_mask   , 'lat': mesh_mask  , 'depth': ANHA12_gridW_files[0], 'data': ANHA12_gridV_files},
             'W': {'lon': mesh_mask   , 'lat': mesh_mask  , 'depth': ANHA12_gridW_files[0], 'data': ANHA12_gridW_files},
             'Pb': {'lon': files_Pb[0], 'lat': files_Pb[0], 'depth': files_Pb[0], 'data': files_Pb}}

dimensions = {'U': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'depthw'},
              'V': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'depthw'},
              'W': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'depthw'},
              'Pb': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'deptht'}}

variables = {'U': 'vozocrtx',
             'V': 'vomecrty',
             'W': 'vovecrtz',
             'Pb': 'dissolpb'}

# needed to provide time as timestamps in order to avoid time origin type issue
fieldset  = FieldSet.from_nemo(filenames, variables, dimensions, timestamps=timestamps, allow_time_extrapolation=True)

#### Classes and functions

In [6]:
# define new particle class for Pb
class PbParticle(JITParticle): 
    dissolvedPb = Variable('dissolpb', initial=fieldset.Pb)  # Variable 'dissolpb' initialised by sampling the dissolved Pb field

In [5]:
def DeleteParticle(particle, fieldset, time):
    print("Particle [%d] lost !! (%g %g %g %g)" % (particle.id, particle.lon, particle.lat, particle.depth, particle.time))
    particle.delete()

In [ ]:
def SamplePb(particle, fieldset, time):
         particle.dissolpb = fieldset.Pb[time, particle.depth, particle.lat, particle.lon]

#### Run

With Pb:

In [ ]:
# When do I want to start tracking the parcels?

release_time  = 0

In [7]:
pset = ParticleSet(fieldset = fieldset, 
                   pclass   = PbParticle, 
                   lon      = parcel_lon, 
                   lat      = parcel_lat, 
                   depth    = parcel_depth,
                   time     = release_time)

output_file = pset.ParticleFile(name="/data/brogalla/run_storage/ocean_parcels/Pb-test2.nc", outputdt=delta(hours=12))

pset.execute(AdvectionRK4_3D + pset.Kernel(SamplePb), 
             runtime     = delta(hours=24*90), 
             dt          = delta(hours=2),
             recovery    = {ErrorCode.ErrorOutOfBounds: DeleteParticle},
             output_file = output_file)

output_file.export()
# dt      --- the timestep of the kernel
# runtime --- the total length of the run

Without Pb: (to check release locations)

In [ ]:
release_time  = 0

In [ ]:
pset = ParticleSet(fieldset = fieldset, 
                   lon      = parcel_lon, 
                   lat      = parcel_lat, 
                   depth    = parcel_depth,
                   time     = release_time)

output_file = pset.ParticleFile(name="/ocean/brogalla/parcels/Pb-chapter/2015-parcel-tracks.nc", outputdt=delta(hours=12))

pset.execute(AdvectionRK4_3D, 
             runtime     = delta(hours=24*90), 
             dt          = delta(hours=2),
             recovery    = {ErrorCode.ErrorOutOfBounds: DeleteParticle},
             output_file = output_file)

output_file.export()
# dt      --- the timestep of the kernel
# runtime --- the total length of the run